In [7]:
from desisim.scripts import quickgen

In [4]:
simargs = ['--brickname', 'lens01',
           '--nspec', '3',
           '--objtype', 'ELG']
args = quickgen.parse(simargs)
args

Namespace(add_SNeIa=False, airmass=1.25, brickname='lens01', config='desi', exptime=None, fibermap=None, frameonly=False, moon_angle=None, moon_phase=None, moon_zenith=None, nspec=3, nstart=0, objtype='ELG', outdir='.', outdir_truth=None, rmagrange_bgs=(15.0, 19.5), seed=0, simspec=None, sne_rfluxratiorange=(0.1, 1.0), spectrograph=None, verbose=False, zrange_bgs=(0.01, 0.4), zrange_elg=(0.6, 1.6), zrange_lrg=(0.5, 1.1), zrange_qso=(0.5, 4.0))

In [8]:
# Generate the spectra.
quickgen.main(args)

INFO:quickgen.py:275:main: Initializing SpecSim with config desi
INFO:quickgen.py:329:main: Generating ELG template
INFO:io.py:606:read_basis_templates: Reading /Users/ioannis/research/projects/desi/spectro/templates/basis_templates/v2.3/elg_templates_v2.0.fits


/usr/local/anaconda3/envs/desi/lib/python3.5/site-packages/astropy/units/quantity.py:892: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


AssertionError: 